In [1]:
import pysam
import pandas as pd

In [2]:
gff_path = '/public/home/mowp/db/Arabidopsis_thaliana/gff3/Arabidopsis_thaliana.TAIR10.46.gff3'

GFF_COLUMNS = ['seqid', 'source', 'type', 'start', 'end', 
               'score', 'strand', 'phase', 'attribute']
gff_df = pd.read_csv(gff_path, sep='\t', comment='#', names=GFF_COLUMNS)

rep_gene_path = '/public/home/lab-zhai/alldata/genome/ath/Araport11/represent.protein.id.txt'
representative_gene_model = pd.read_csv(rep_gene_path, comment='#', header=None, sep='\t')
representative_gene_model = set(representative_gene_model[1])

/public/home/mowp/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
cds_df = gff_df[gff_df['type'].isin(['CDS'])]

In [6]:
cds_df.iloc[0][8]

'ID=CDS:AT1G01010.1;Parent=transcript:AT1G01010.1;protein_id=AT1G01010.1'

In [7]:
def get_transcript_id(attr):
    '''get transcript_id from attribute
    '''
    for item in attr.split(';'):
        key, value = item.split('=')
        if key == 'Parent':
            return value.replace('transcript:', '')

In [8]:
cds_df['transcript_id'] = cds_df['attribute'].map(get_transcript_id)

/public/home/mowp/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
representative_cds_model =  cds_df[cds_df['transcript_id'].isin(representative_gene_model)]

In [33]:
curr_transcript_id = ''
start_list, end_list = [], [] 
n = 0
for item in representative_cds_model.itertuples():
    if curr_transcript_id == '':
        curr_transcript_id = item.transcript_id
    if curr_transcript_id == item.transcript_id:
        start_list.append(item.start)
        end_list.append(item.end)
        curr_item = item
    else:
        print(f'{curr_item.seqid}\t{min(start_list)-1}\t{max(end_list)}\t{curr_item.transcript_id}\t0\t{curr_item.strand}')
        start_list.clear()
        end_list.clear()
        curr_transcript_id = item.transcript_id
        start_list.append(item.start)
        end_list.append(item.end)
        n += 1

1	3759	5630	AT1G01010.1	0	+
1	6914	8666	AT1G01020.1	0	-
1	11863	12940	AT1G01020.1	0	-
1	23518	31079	AT1G01040.2	0	+
1	31381	32670	AT1G01050.1	0	-
1	33991	37061	AT1G01060.1	0	-
1	38897	40877	AT1G01070.1	0	-
1	45502	46789	AT1G01080.2	0	-
1	47704	49166	AT1G01090.1	0	-
1	50283	50954	AT1G01100.1	0	-
1	52238	54494	AT1G01110.2	0	+


In [27]:
i.start

3760